In [1]:
import numpy as np

In [12]:
prevFrameMidPointsToActionDict = None
prevFrameMidPoints = None
threshold = 30

In [5]:
def getMidPoint(bbox):
    '''
    Helper function of conertBboxsToMidPoint()
    '''
    p1, p2 = bbox[0] , bbox[1]
    rowMid = (p1[0] + p2[0])/2
    widthMid = (p1[1] + p2[1])/2
    return (rowMid , widthMid)

def convertBboxsToMidPoint(bboxs):
    '''
     Parameter
     ----------
     bboxs : A 2D list containing the two coordinates of box, p1(left,top) and p2(right,bottom)
     [ [p1 , p2] , ... ] (from getBoxCoord)
     
     Return
     -------------
     midPointList : A List containig tuple of mid-point coord for each box
     Example : [(mX,mY), ....]
    '''
    midPointList =  list(map(getMidPoint , bboxs))
    return midPointList


In [7]:
def getDistanceMidToMid(midPoint1, midPoint2):
    '''
    Helper function of getDistacneMidToMids()
    '''
    dist = np.sqrt((midPoint1[0] - midPoint2[0])**2 + (midPoint1[1] - midPoint2[1])**2)
    return dist

def getDistanceMidToMids(midPoint1,midPoints):
    '''
    This function compares the distacne of 1 mid point to list of midpoints
    
    Parameter 
    -----------
    midPoint1 : A list containing midPoint coordintes [mx, my]
    
    Return
    -------
    distances : Distance of midPoint1 to all other midpoints in midPoints
    Example : [3 ,4 ,5 ....]
    
    '''
    distances = [getDistanceMidToMid(midPoint1,midPoint2) for midPoint2 in midPoints]
    return distances



In [8]:
def getAllMidPointsDistance(midPointList):
    '''
    This function , for each midpoint in midPointList, it find the distance to other midpoints in the 
    previous frame
    
    Parameter 
    ---------
    midPointList  :  A list contianing midPoints
    Example : [(mx,my)....(mx,my)]
    
    Return 
    ----------
    midPointsDistances : A 2D list containing distance to all the other midpoints 
    Example : [ [1,2,3] , [4,5,9] , ... ]
    
    '''
    global prevFrameMidPoints
    midPointsDistances = [getDistanceMidToMids(midPoint1 , prevFrameMidPoints) for midPoint1 in midPointList]
    return midPointsDistances

In [17]:
def getClosestMidPoint(midPointDistance):
    '''
    Helper function of track()
    '''
    global prevFrameMidPoints
    closestMidPoint = prevFrameMidPoints[np.argmin(midPointDistance)]
    distanceToClosestMidPoint = np.min(midPointDistance)
    return closestMidPoint , distanceToClosestMidPoint


def track(bboxs):
    '''
    This function will return action for each detectect pose/bbox.
    It will do this by finding the closest bounding box in the last frame.
    
    Parameter
    ----------
    bboxs  : A list containing bboxs. Where each box contain 2 points of the box
    Example : [[p1,p2] , ....[]..]
    
    Return 
    -----------
    bboxsAction : A dict containing estimated action class for each pose/box in bboxs 
    None is returned if no closest midpoint is found
    Example : [(midpoint): 'Shoot',(midPoint) : None .....] (order is kept)
    '''
    
    global prevFrameMidPointsToActionDict
    bboxsAction = {}
    
    midPointsList = convertBboxsToMidPoint(bboxs)
    midPointsDistances = getAllMidPointsDistance(midPointsList)
    for midPointDistance in midPointsDistances:
        closestMidPoint , distanceToClosestMidPoint = getClosestMidPoint(midPointDistance)
        if distanceToClosestMidPoint < threshold:
            action =  prevFrameMidPointsToActionDict[closestMidPoint]
        else:
            action = None
        bboxsAction[closestMidPoint] = action
    
    
    return bboxsAction


In [24]:
def setTracker(bboxs,classes):
    '''
    This function sets the list of midpoints and dict that contains midPoints and classes.
    
    Parameter
    ----------
    bboxs  : A list containing bboxs. Where each box contain 2 points of the box
    Example : [[p1,p2] , ....[]..]
    
    classes :  A list of action classes for the bboxes. It should not contain any None Value
    Example : ['Shoot',.......]
    '''
    global prevFrameMidPoints , prevFrameMidPointsToActionDict
    prevFrameMidPoints = convertBboxsToMidPoint(bboxs)
    prevFrameMidPointsToActionDict = {midPoint : classes[i][0] for i,midPoint in enumerate(prevFrameMidPoints)}

#Test

In [18]:
bboxs = [[(97, 311), (311, 586)]]

In [19]:
mList = convertBboxsToMidPoint(bboxs)

In [22]:
 #getAllMidPointsDistance(mList)

TypeError: 'NoneType' object is not iterable